In [79]:
import zipfile
import os
import pandas as pd
import seaborn as sns
import scipy.stats as st
import pingouin as pg

In [80]:
zip_path = os.path.expanduser('~/dataset.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(r'ab_data.csv') as csv_file:
        df = pd.read_csv(csv_file)
        

In [81]:
#os.listdir()
#os.getcwd()

In [82]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [84]:
df.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

In [85]:
df['timestamp'] = pd.to_datetime(df.timestamp)

In [86]:
df.group.value_counts()

group
treatment    147276
control      147202
Name: count, dtype: int64

In [87]:
df.converted.value_counts()

converted
0    259241
1     35237
Name: count, dtype: int64

In [88]:
df.loc[(df.group == 'control') & (df.landing_page == 'new_page')]

,user_id,timestamp,group,landing_page,converted
22,767017,2017-01-12 22:58:14.991443,control,new_page,0
240,733976,2017-01-11 15:11:16.407599,control,new_page,0
490,808613,2017-01-10 21:44:01.292755,control,new_page,0
846,637639,2017-01-11 23:09:52.682329,control,new_page,1
850,793580,2017-01-08 03:25:33.723712,control,new_page,1
...,...,...,...,...,...
293894,741581,2017-01-09 20:49:03.391764,control,new_page,0
293996,942612,2017-01-08 13:52:28.182648,control,new_page,0
294200,928506,2017-01-13 21:32:10.491309,control,new_page,0
294253,886135,2017-01-06 12:49:20.509403,control,new_page,0


In [89]:
df.loc[(df.group == 'treatment') & (df.landing_page == 'old_page')]

,user_id,timestamp,group,landing_page,converted
308,857184,2017-01-20 07:34:59.832626,treatment,old_page,0
327,686623,2017-01-09 14:26:40.734775,treatment,old_page,0
357,856078,2017-01-12 12:29:30.354835,treatment,old_page,0
685,666385,2017-01-23 08:11:54.823806,treatment,old_page,0
713,748761,2017-01-10 15:47:44.445196,treatment,old_page,0
...,...,...,...,...,...
293773,688144,2017-01-16 20:34:50.450528,treatment,old_page,1
293817,876037,2017-01-17 16:15:08.957152,treatment,old_page,1
293917,738357,2017-01-05 15:37:55.729133,treatment,old_page,0
294014,813406,2017-01-09 06:25:33.223301,treatment,old_page,0


In [90]:
df_corrected = df.loc[~((df.group == 'treatment') & (df.landing_page == 'old_page') | (df.group == 'control') & (df.landing_page == 'new_page'))]

In [91]:
df_corrected.shape

(290585, 5)

In [103]:
df_corrected.user_id.duplicated().sum()

np.int64(1)

In [104]:
df_corrected.user_id.nunique()

290584

In [107]:
df_cleared = df_corrected.drop_duplicates(subset = 'user_id')

In [108]:
df_cleared.shape

(290584, 5)

In [109]:
df_ct = pd.crosstab(df_corrected.group, df_corrected.converted)
df_ct

converted,0,1
group,,
control,127785,17489
treatment,128047,17264


In [110]:
st.chi2_contingency(df_ct)

Chi2ContingencyResult(statistic=np.float64(1.7053502645115002), pvalue=np.float64(0.19158976298516003), dof=1, expected_freq=array([[127899.7125385,  17374.2874615],
       [127932.2874615,  17378.7125385]]))